In [9]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import napari
import numpy as np
import os

from scribbles_creator import *
from cellpose_data_handler import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
gt = np.zeros((512, 512))
create_even_scribbles(gt)

Using square size of 17 pixels.


## Define where the images are located

In [14]:
folder_path = "./cellpose_train_imgs/"

## Create ground truth for pixel classification from image mask (instance segmentation)

In [ ]:
# for img_num in range(0, 540):
#     create_cellpose_gt(folder_path, img_num, save_res=True, show_res=False)

## Create scribble annotation from an image mask of the cell pose dataset

In [ ]:
# mode = "all"
# bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:5]

# percentages = {}
# for bin in bins:
#     percentages[bin] = []
#     for img_num in range(0, 10):
#         for s in suff:
#             print(f"IMG {img_num}: {bin}, {s}")
#             scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, mode=mode, save_res=True, suff=s, show_res=False, print_steps=False)
#             percentages[bin].append(perc_labelled)
#             print("\n")

Same cell but for tests...

In [17]:
mode = "all"
bins = [1.55] #[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
margin = 0.75
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]
rel_scribble_len = False
scribble_width = 5 #1
scribbles_seed = 1

percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num in range(0, 540, 20): #[200, 50, 272, 351, 493]:#, 383, 419]:#range(0, 10):
        for s in suff:
            print(f"IMG {img_num}: {bin}, {s}")
            np.random.seed(scribbles_seed)
            scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, margin=margin, mode=mode, save_res=False, suff=s, show_res=False, print_steps=False,
                                                                rel_scribble_len=rel_scribble_len, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"annotation: {perc_labelled:.2f}%")
            print("\n")

IMG 0: 1.55, a
annotation: 9.67%


IMG 20: 1.55, a
annotation: 9.62%


IMG 40: 1.55, a
annotation: 9.45%


IMG 60: 1.55, a
annotation: 10.23%


IMG 80: 1.55, a
annotation: 9.99%


IMG 100: 1.55, a
annotation: 9.98%


IMG 120: 1.55, a
annotation: 9.91%


IMG 140: 1.55, a
annotation: 9.84%


IMG 160: 1.55, a
annotation: 9.91%


IMG 180: 1.55, a
annotation: 10.11%


IMG 200: 1.55, a
annotation: 9.41%


IMG 220: 1.55, a
annotation: 9.84%


IMG 240: 1.55, a
annotation: 9.89%


IMG 260: 1.55, a
annotation: 10.07%


IMG 280: 1.55, a
annotation: 10.89%


IMG 300: 1.55, a
annotation: 10.39%


IMG 320: 1.55, a
annotation: 9.96%


IMG 340: 1.55, a
annotation: 10.57%


IMG 360: 1.55, a
annotation: 9.22%


IMG 380: 1.55, a
annotation: 10.09%


IMG 400: 1.55, a
annotation: 10.34%


IMG 420: 1.55, a
annotation: 10.08%


IMG 440: 1.55, a
annotation: 10.29%


IMG 460: 1.55, a
annotation: 10.11%


IMG 480: 1.55, a
annotation: 11.19%


IMG 500: 1.55, a
annotation: 11.14%


IMG 520: 1.55, a
annotation: 10

Quick report of labelling percentages:

In [18]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 1.55: mean = 10.1047, std = 0.4705, range = 9.2250 - 11.1871


Show the last created scribbles

In [44]:
napari.view_labels(scribbles)

Viewer(camera=Camera(center=(0.0, 255.5, 255.5), zoom=0.5826171875, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=10, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 512.0, 1.0), (0.0, 512.0, 1.0)), current_step=(255, 255), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Labels layer 'scribbles' at 0x286471de3d0>], help='use <1> for activate the label eraser, use <2> for activate the paint brush, use <3> for activate the fill bucket, use <4> for pick mode', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000002865F038280>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

## Load and show scribbles and ground truth from saved files

In [ ]:
# Show the image, ground truth and the scribble annotation in a napari viewer
img_num = 0
mode = "all"
bin = 0.1
suff = "a"
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, suff=suff)
v = napari.Viewer()
v.add_image(img_data["img"], name="image")
v.add_labels(img_data["gt"], name="ground truth")
v.add_labels(img_data["scribbles"], name="scribbles")

## When we have performed segmentation in the Napari plugin, we can calculate its accuracy

In [ ]:
# segmentation = v.layers['segmentation'].data
# acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
# print(f"Accuracy (pixels correctly segmented): {acc}%")

# TESTS

In [ ]:
from matplotlib import pyplot as plt

for i in range(1000):
    start = (225,25)
    row = [True] * 300
    mask = np.array([row] * 300)
    mask[50:255, 150:255] = False
    mask[50:100, 10:100] = False
    mask[200:250, 50:100] = False
    res = point_to_edge(start, mask)
# print(res)
# res_int = np.array(res, dtype=np.uint16)
# print(np.argwhere(res == False).shape)
# plt.imshow(res)

plt.imshow(np.logical_or(res, np.logical_not(mask)))
plt.colorbar()


In [ ]:
from matplotlib import pyplot as plt

row = [False] * 300
mask = np.array([row] * 300)
mask[50:73,140] = True
mask[52,143:160] = True
mask_changed = mask.copy()
plt.imshow(mask)

max_pix = 5
coord = np.where(mask)
# print(coord)

mask_changed[coord[0][max_pix:], coord[1][max_pix:]] = False
# for i in range(max_pix, len(coord[0])):
#     print(i)
#     print(coord[0][i], coord[1][i])
#     mask_changed[coord[0][i], coord[1][i]] = False

print("num pix:", np.sum(mask_changed))
plt.imshow(mask_changed)